<a href="https://colab.research.google.com/github/oimartin/Older-and-Wiser/blob/main/medicare_population_65_plus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Injest Data
* Source - Centers for Medicare and Medicaid Services
* https://data.cms.gov/summary-statistics-on-beneficiary-enrollment/medicare-and-medicaid-reports/medicare-newly-enrolled

In [96]:
# !pip install -U kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [101]:
import pandas as pd
import numpy as np
import plotly.express as px
import kaleido

# Area Data

## Filter

In [2]:
# load data
total_area_20 = pd.read_csv('https://raw.githubusercontent.com/oimartin/Older-and-Wiser/main/data/2020_total_area_medicare_enrolled.csv')
state_pop = pd.read_csv('https://raw.githubusercontent.com/oimartin/Older-and-Wiser/main/data/us_census_2020_population.csv')
state_abbrev = pd.read_csv('https://raw.githubusercontent.com/jasonong/List-of-US-States/master/states.csv')

In [3]:
# function to modify Medicare area excel files

def area_df(df):
  df.drop(columns='Unnamed: 13', inplace=True)
  df.iloc[:,1:] = df.iloc[:, 1:].replace('  ', np.nan, regex=True)
  df.dropna(axis=0,inplace=True)
  df.columns = df.iloc[0]

  clean = df.iloc[1:,0:2]
  clean.iloc[:,:] = clean.replace(',', '', regex=True)
  clean.iloc[:,:] = clean.replace('*', 0)
  clean.iloc[:,1] = clean.iloc[:,1].astype(int)
  return clean

In [4]:
# Check shape
clean_area_20 = area_df(total_area_20)
display(clean_area_20.shape)

(60, 2)

In [5]:
# function to modify US population excel file
def pop_df(df):
  df = df.iloc[:, 0:2].dropna(axis=0)
  df.iloc[:,:] = df.replace(',', '', regex=True)
  df['Total:'] = df['Total:'].astype(int)

  return df

In [6]:
# Clean and add state population to clean_area_20 df
clean_pop = pop_df(state_pop)
clean_area_20['State_Pop'] = 1
clean_area_20.iloc[2:54,2] = clean_pop['Total:']
clean_area_20['A.B_aged_state_pop_porp'] = clean_area_20.iloc[2:54,1] / clean_area_20.iloc[2:54,2]
clean_area_20['AB_aged_medicare_aged_prorp'] = clean_area_20.iloc[:,1]/clean_area_20.iloc[0,1]

In [7]:
clean_area_20.head()

2,Area of Residence,Part A and/or Part B Total,State_Pop,A.B_aged_state_pop_porp,AB_aged_medicare_aged_prorp
4,All Areas,3793542,1,NaN,1.000000
5,United States,3739508,1,NaN,0.985756
7,Alabama,58719,5024279,0.011687,0.015479
8,Alaska,8263,733391,0.011267,0.002178
9,Arizona,78342,7151502,0.010955,0.020651


In [8]:
# Create regions
clean_area_20['Region'] = clean_area_20['Area of Residence']
clean_area_20['Region'].replace(['Connecticut','Maine','Massachusetts',
                                 'New Hampshire','Rhode Island', 'Vermont'],
                                'New England',inplace=True)

clean_area_20['Region'].replace(['New Jersey','New York','Pennsylvania'],
                                'Middle Atlantic',inplace=True)

clean_area_20['Region'].replace(['Indiana','Illinois', 'Michigan',
                                 'Ohio', 'Wisconsin'],
                                'East North Central',inplace=True)

clean_area_20['Region'].replace(['Iowa', 'Nebraska', 'Kansas', 'North Dakota',
                                 'Minnesota', 'South Dakota', 'Missouri'],
                                'West North Central',inplace=True)

clean_area_20['Region'].replace(['Delaware', 'District of Columbia', 'Florida',
                                 'Georgia', 'Maryland', 'North Carolina',
                                 'South Carolina', 'Virginia', 'West Virginia'],
                                'South Atlantic',inplace=True)

clean_area_20['Region'].replace(['Arkansas', 'Louisiana', 'Oklahoma', 
                                 'Texas'],
                                'West South Central',inplace=True)

clean_area_20['Region'].replace(['Alabama', 'Kentucky', 'Mississippi', 
                                 'Tennessee'],
                                'East South Central',inplace=True)

clean_area_20['Region'].replace(['Arizona', 'Colorado', 'Idaho','New Mexico',
                                 'Montana', 'Utah', 'Nevada', 'Wyoming'],
                                'Mountain',inplace=True)

clean_area_20['Region'].replace(['Alaska', 'California', 'Hawaii', 'Oregon',
                                 'Washington'],
                                'Pacific',inplace=True)

clean_area_20['Region'].replace(['Puerto Rico', 'Virgin Islands',
                                 'American Samoa', 'Guam', 'Northern Mariana Islands'],
                                'Territories',inplace=True)


In [9]:
# Only select regions and not entire US
regions = clean_area_20[(clean_area_20['Region'] != 'All Areas') &
              (clean_area_20['Region'] != 'United States') &
              (clean_area_20['Region'] != 'Unknown')].groupby('Region').sum().reset_index()

In [10]:
# Check regions df
regions['Region'].replace({'Foreign Countries and Other Outyling Areas': 'Outside US'}, inplace=True)
regions

2,Region,Part A and/or Part B Total,State_Pop,A.B_aged_state_pop_porp,AB_aged_medicare_aged_prorp
0,East North Central,560799,47368533,0.059580,0.147830
1,East South Central,227063,19402234,0.046933,0.059855
2,Outside US,20349,1,0.000000,0.005364
3,Middle Atlantic,489856,42492943,0.034662,0.129129
4,Mountain,270236,24919150,0.091681,0.071236
5,New England,185837,15116205,0.078653,0.048988
6,Pacific,567234,53669422,0.056851,0.149526
7,South Atlantic,765417,66089734,0.102240,0.201768
8,Territories,33684,3285878,0.009289,0.008879
9,West North Central,255159,21616921,0.083567,0.067261


## Regions

### Regions named


> Northeast
>>New England
1. Connecticut
2. Maine
3. Massachusetts
4. New Hampshire
5. Rhode Island
6. Vermont

>> Middle Atlantic
7. New Jersey
8. New York
9. Pennsylvania

> Midwest
>>East North Central
1. Indiana
2. Illinois
3. Michigan
4. Ohio
5. Wisconsin

>>West North Central
6. Iowa
7. Nebraska
8. Kansas
9. North Dakota
10. Minnesota
11. South Dakota
12. Missouri

> South
>> South Atlantic
1. Delaware
2. District of Columbia
3. Florida
5. Georgia
6. Maryland
8. North Carolina
9. South Carolina
10. Virginia
11. West Virginia

>> East South Central
12. Alabama
13. Kentucky
14. Mississippi
15. Tennessee

>> West South Central
16. Arkansas
17. Louisiana
18. Oklahoma
19. Texas

>West
>> Mountain
1. Arizona
2. Colorado
3. Idaho
4. New Mexico
5. Montana
6. Utah
7. Nevada
8. Wyoming

>> Pacific
9. Alaska
10. California
11. Hawaii
12. Oregon
13. Washington




### Region Graphs

In [11]:
fig = px.bar(regions.sort_values('Part A and/or Part B Total'), x="Region",
             y='Part A and/or Part B Total', color="A.B_aged_state_pop_porp",
             color_continuous_scale=['#ffefd6', '#db5022', '#44131a'])
fig.update_layout(template="plotly_white") 
fig.update_coloraxes(colorbar_orientation='h')
fig.show()

In [48]:
states = clean_area_20[(clean_area_20['Region'] != 'All Areas') &
              (clean_area_20['Region'] != 'United States') &
              (clean_area_20['Region'] != 'Unknown')].copy()

In [49]:
states['Region_porp'] = states['Region'].replace({'East North Central': regions.iloc[0,3],
                                                  'East South Central': regions.iloc[1,3],
                                                  'Foreign Countries and Other Outyling Areas':regions.iloc[2,3],
                                                  'Middle Atlantic':regions.iloc[3,3],
                                                  'Mountain':regions.iloc[4,3],
                                                  'New England':regions.iloc[5,3],
                                                  'Pacific':regions.iloc[6,3],
                                                  'South Atlantic':regions.iloc[7,3],
                                                  'Territories':regions.iloc[8,3],
                                                  'West North Central':regions.iloc[9,3],
                                                  'West South Central':regions.iloc[10,3]}).astype('float64')

In [ ]:
just_states = states.iloc[:-6, :].copy()
just_states.reset_index(inplace=True)
just_states.drop(columns='index',inplace=True)

state_abbrev.sort_values(by='State', inplace=True)
state_abbrev.reset_index(inplace=True)
state_abbrev.drop(columns='index', inplace=True)

just_states['Abbrev'] = state_abbrev['Abbreviation'].copy()

In [103]:
fig = px.choropleth(just_states,
                    locations='Abbrev', 
                    locationmode="USA-states", 
                    scope="usa",
                    color='Region_porp',
                    color_continuous_scale=['#ffefd6', '#db5022', '#44131a'],
                    range_color=(0,.1))
fig.update_layout(margin = dict(t=10, l=10, r=10, b=10))

fig.show()
cholorpath_map = fig.to_image(format="png")

ValueError: ignored

# Demo data

## takein

In [ ]:
total_demo_15 = pd.read_csv('https://raw.githubusercontent.com/oimartin/Older-and-Wiser/medicare/data/2015_total_demo_medicare_enrolled.csv')
total_demo_16 = pd.read_csv('https://raw.githubusercontent.com/oimartin/Older-and-Wiser/medicare/data/2016_total_demo_medicare_enrolled.csv')
total_demo_17 = pd.read_csv('https://raw.githubusercontent.com/oimartin/Older-and-Wiser/medicare/data/2017_total_demo_medicare_enrolled.csv')
total_demo_18 = pd.read_csv('https://raw.githubusercontent.com/oimartin/Older-and-Wiser/medicare/data/2018_total_demo_medicare_enrolled.csv')
total_demo_19 = pd.read_csv('https://raw.githubusercontent.com/oimartin/Older-and-Wiser/medicare/data/2019_total_demo_medicare_enrolled.csv')
total_demo_20 = pd.read_csv('https://raw.githubusercontent.com/oimartin/Older-and-Wiser/medicare/data/2020_total_demo_medicare_enrolled.csv')

In [ ]:
display(total_demo_15.shape,
        total_demo_16.shape,
        total_demo_17.shape,
        total_demo_18.shape,
        total_demo_19.shape,
        total_demo_20.shape)

In [ ]:
def demo_df(df, year):
  df.drop(columns=['Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7',
                            'Unnamed: 8', 'Unnamed: 9'],inplace=True)
  df.dropna(axis=0,inplace=True)
  df.columns = df.iloc[0]
  clean = df.iloc[1:, :]
  cleaner = clean.replace(',', '', regex=True)
  v_clean = cleaner[(clean['Demographic Characteristic'] != 'Age') &
      (clean['Demographic Characteristic'] != 'Sex') &
      (clean['Demographic Characteristic'] != 'Race')]
  for column in list(v_clean.columns)[1:-1]:
    v_clean[column] = v_clean[column].astype(int)
  v_clean['Year'] = year

  return v_clean

In [ ]:
clean_demo_15 = demo_df(total_demo_15, '2015')
clean_demo_16 = demo_df(total_demo_16, '2016')
clean_demo_17 = demo_df(total_demo_17, '2017')
clean_demo_18 = demo_df(total_demo_18, '2018')
clean_demo_19 = demo_df(total_demo_19, '2019')
clean_demo_20 = demo_df(total_demo_20, '2020')

In [ ]:
display(clean_demo_15.shape,
        clean_demo_16.shape,
        clean_demo_17.shape,
        clean_demo_18.shape,
        clean_demo_19.shape,
        clean_demo_20.shape)

## Reorganize demo data

In [ ]:
index_65_2015 = clean_demo_15[clean_demo_15['Demographic Characteristic'] == '65-74 Years'].iloc[0,1]

In [ ]:
def choose_65(df):
  new_df = df[df['Demographic Characteristic'] == '65-74 Years']
  return new_df

In [ ]:
d65_20 = choose_65(clean_demo_20)
d65_19 = choose_65(clean_demo_19)
d65_18 = choose_65(clean_demo_18)
d65_17 = choose_65(clean_demo_17)
d65_16 = choose_65(clean_demo_16)
d65_15 = choose_65(clean_demo_15)

d65_all = pd.concat([d65_15, d65_16, d65_17, d65_18, d65_19, d65_20])

In [ ]:
def choose_75_plus(df):
  new_df = df[(df['Demographic Characteristic'] == '75-84 Years') |
              (df['Demographic Characteristic'] == '85-94 Years') |
              (df['Demographic Characteristic'] == '95 Years and Over')]
  
  return new_df

In [ ]:
d75_plus_20 = choose_75_plus(clean_demo_20)
d75_plus_19 = choose_75_plus(clean_demo_19)
d75_plus_18 = choose_75_plus(clean_demo_18)
d75_plus_17 = choose_75_plus(clean_demo_17)
d75_plus_16 = choose_75_plus(clean_demo_16)
d75_plus_15 = choose_75_plus(clean_demo_15)

d75_plus_all = pd.concat([d75_plus_15, d75_plus_16,
                          d75_plus_17, d75_plus_18,
                          d75_plus_19, d75_plus_20])

d75_plus_all

## Graphs

### Bar graphs

In [ ]:
fig = px.bar(d75_plus_all, x="Demographic Characteristic", y="Part A and/or Part B", color="Year", barmode="group")
fig.show()

In [ ]:
fig = px.bar(d65_all, x="Demographic Characteristic", y="Part A and/or Part B", color="Year", barmode="group")
fig.show()

#### Box Whisker Plot

In [ ]:
fig = px.box(d65_all, y="Demographic Characteristic", x="Part A and/or Part B",
             orientation='h')
fig.show()

In [ ]:
sns.boxplot(data=d65_all, y='Demographic Characteristic', x="Part A and/or Part B",
            orient='h')

In [ ]:
fig = px.box(d75_plus_all, y="Demographic Characteristic", x="Part A and/or Part B",
             orientation='h', color='Demographic Characteristic')
fig.show()